In [ ]:
%load_ext autoreload
%autoreload 2


import numpy as np
import time, os, sys

import skimage.io
import matplotlib.pyplot as plt
plt.style.use('dark_background')
import matplotlib as mpl
%matplotlib inline
mpl.rcParams['figure.dpi'] = 300

from scipy.ndimage import binary_closing,grey_closing
from skimage import measure
import omnipose
import ncolor 
from omnipose.utils import sinebow

from scipy.ndimage import gaussian_filter
from omnipose.utils import normalize99, rescale
from scipy.ndimage import gaussian_filter

def localnormalize(im,sigma1=2,sigma2=3):
    im = normalize99(im)
    blur1 = gaussian_filter(im,sigma=sigma1)
    num = im - blur1
    blur2 = gaussian_filter(num*num, sigma=sigma2)
    den = np.sqrt(blur2)
    
    return normalize99(num/den+1e-8)

def cyclic_perm(a):
    n = len(a)
    b = [[a[i - j] for i in range(n)] for j in range(n)]
    return b

# Make a function that extracts the file name 
from pathlib import Path
def getname(path,suffix=''):
    return os.path.splitext(Path(path).name)[0].replace(suffix,'')

import tifffile
from omnipose import core
import fastremap
import scipy
from cellpose_omni import io


In [ ]:
# !pip install aicsimageio
# !pip install bioformats_jar
from aicsimageio import AICSImage

In [ ]:
# save0 = '/home/kcutler/DataDrive/3D_BBBC/BBBC027'
basedir = '/home/kcutler/DataDrive/3D_BBBC/BBBC027'
savedir = []
cpdir = []
ext = '.tif'
for t in ['test','train']:
    sd = os.path.join(basedir,t)
    io.check_dir(sd)
    savedir.append(sd)
    sd = os.path.join(basedir,'cellpose',t)
    io.check_dir(sd)
    cpdir.append(sd)
savedir,cpdir

In [ ]:
for number in range(30):
    name = "%04d" % (number,)
    img = AICSImage(os.path.join(save0,'image-final_'+name+'.ics')).data.squeeze()
    mask =  AICSImage(os.path.join(save0,'image-labels_'+name+'.ics')).data.squeeze()
    labels, _ = scipy.ndimage.label(mask)
    labels = fastremap.refit(labels)
    if number%3:
        sd = savedir[0]
    else:
        sd = savedir[1]
        
    tifffile.imwrite(os.path.join(sd,name+ext),img)
    tifffile.imwrite(os.path.join(sd,name+'_masks'+ext),labels)

In [ ]:
d = 3
idx = np.arange(d)
c = np.array([0]*(d-2)+[1]*2)
cyclic_perm(c),idx,c
ext = '.tif'
a = 'zyx'
# for basedir,save0 in zip(expdir,cpdir):
for basedir,save0 in zip([savedir[1]],[cpdir[1]]): #only do train 

    mask_filter = '_masks'
    img_names = io.get_image_files(basedir,mask_filter)
    mask_names,_ = io.get_label_files(img_names, mask_filter)
    for p1,p2 in zip(img_names,mask_names):
        img = tifffile.imread(p1)
        mask = tifffile.imread(p2)
        s = img.shape
        name = getname(p1)
        for inds,i in zip(cyclic_perm(c),idx):
            for k in range(s[i]):
                slc = tuple([slice(-1) if i else k for i in inds])
                suffix = '_'+a[i]+'{:03d}'.format(k)
                l = omnipose.utils.format_labels(mask[slc],clean=True)
                if np.any(l): # only keep slices that have masks
                    tifffile.imwrite(os.path.join(save0,name+suffix+ext),np.uint16(omnipose.utils.rescale(img[slc])*(2**16-1)))
                    tifffile.imwrite(os.path.join(save0,name+suffix+'_masks'+ext),l)

# Evaluate models

In [ ]:
imgs = []
masks_gt = []
#test files now
mask_filter = '_masks'
img_names = io.get_image_files(savedir[0],mask_filter)
mask_names,_ = io.get_label_files(img_names, mask_filter)
n = [0]
for p1,p2 in zip([img_names[i] for i in n],[mask_names[i] for i in n]):
    imgs.append(tifffile.imread(p1))
    masks_gt.append(tifffile.imread(p2))

In [ ]:
c = 0
for mgt in masks_gt:
    c+=len(fastremap.unique(mgt))
print(c)

In [ ]:
modeldir = '/home/kcutler/DataDrive/3D_BBBC/BBBC027/cellpose/train/models/cellpose_residual_on_style_on_concatenation_off_train_2022_05_10_03_44_47.268923_epoch_499'
omni = False
rescale = False
diam_mean = 0
from cellpose_omni import models, core
use_GPU = core.use_gpu()
model = models.CellposeModel(gpu=use_GPU, pretrained_model=modeldir, net_avg=False, 
                             diam_mean=diam_mean)

In [ ]:
net_avg = 0
verbose = 0 
tile = 0
chans = [0,0]
compute_masks = True
flow_threshold = 0.0
# n = [0,1,2,3,4,5]
n = [0]
rescale = None
# slc = tuple([slice(0,129)]*3)
slc = slice(None)
slc = tuple([slice(0,129),slice(100,300),slice(500,700)])


In [ ]:
masks_cp, flows_cp, _ = model.eval([imgs[i][slc] for i in n],channels = chans,rescale=rescale, net_avg=net_avg,
                                  transparency=True, flow_threshold=flow_threshold, verbose=verbose, tile=tile,
                                  compute_masks=compute_masks, do_3D=True, omni=False)

In [ ]:
# now do Omnipose
import torch
modeldir = '/home/kcutler/DataDrive/3D_BBBC/BBBC027/train/models/cellpose_residual_on_style_on_concatenation_off_omni_train_2022_05_03_22_16_13.356647_epoch_2601'
# modeldir = '/home/kcutler/DataDrive/3D_BBBC/BBBC027/train/models/cellpose_residual_on_style_on_concatenation_off_omni_train_2022_05_03_22_16_13.356647_epoch_2451'

dim = 3
nclasses = dim+2
nchan = 1
omni = 1
rescale = False
diam_mean = 0
from cellpose_omni import models, core
use_GPU = core.use_gpu()
model = models.CellposeModel(gpu=use_GPU, pretrained_model=modeldir, net_avg=False, 
                             diam_mean=diam_mean, nclasses=nclasses, dim=dim, nchan=nchan, omni=omni)#/',device=torch.device('cuda'))#,checkpoint=True,device=torch.device('cuda:1'))

In [ ]:
mask_threshold = -5 
diam_threshold = 12
net_avg = 0
cluster = 0 
verbose = 1
tile = 1
chans = None
compute_masks = 1
resample=False
rescale=None
omni=True
flow_factor = 10.0
masks_om, flows_om, _ = model.eval([imgs[i][slc] for i in n],channels=None,rescale=rescale,mask_threshold=mask_threshold,net_avg=net_avg,
                                  transparency=True,flow_threshold=flow_threshold,omni=omni,resample=resample,verbose=verbose,
                                  diam_threshold=diam_threshold,cluster=cluster,tile=tile,
                                  compute_masks=compute_masks, flow_factor=flow_factor, batch_size=1)

In [ ]:
masks = [masks_cp,masks_om]
flows = [flows_cp,flows_om]
names = ['cellpose','omnipose']
np.array(masks).shape

In [ ]:
# now quanititative comparison 
# !pip install stardist
from stardist import matching
a = 0.5
b = 1.0
thresh =np.arange(a,b,(b-a)/100)
fig = plt.figure()
c = ['r','g']
for i,name in enumerate(names):
    met =[matching.matching(mgt[slc],m, thresh=thresh) for mgt,m in zip(masks_gt, masks[i])]
    a = np.array([[met[k][i].accuracy for i in range(len(thresh))] for k in range(len(met))]).T

    np.save(os.path.join(basedir,name+'_3D_plants_individual'),a)
    a = np.mean(a,axis=1)
    plt.plot(thresh,a, label=name,c=c[i])#+['masks_cp'])
    np.save(os.path.join(basedir,name+'cellpose_3D_plants'),a)
    plt.legend()

In [ ]:
cmap = mpl.cm.get_cmap('viridis')
for j in n:
    for i,name in enumerate(names):
        print(name,i)
        img = imgs[j][slc]
        mgt = masks_gt[j][slc]
        msk = masks[i][j]
        dt = flows[i][j][2]
        bd = flows[i][j][4]
        frgb = flows[i][j][0]
        k = img.shape[0]//3 # middle slice 
        fig = plt.figure()
        plt.imshow(np.hstack(tuple(map(cmap,map(omnipose.utils.rescale,[img[k],dt[k], bd[k],
                                                                        ncolor.label(msk[k]),
                                                                        ncolor.label(mgt[k])])))
                             +(frgb[k]/255,)))
        plt.axis('off')

In [ ]:
import napari
k = 0
viewer = napari.view_image(imgs[k][slc], name='cells')
viewer.add_labels(masks_gt[k][slc], name='labels_gt',visible=False)
viewer.add_image(flows[0][k][0], name=names[0]+'_flows',visible=False)
viewer.add_labels(masks[0][k], name=names[0],visible=False)
viewer.add_image(flows[1][k][0], name=names[1]+'_flows',visible=False)

viewer.add_labels(masks[1][k], name=names[1],visible=False)

In [ ]:
mu = flows_om[0][1]
dt = flows_om[0][2]
p = flows_om[0][3]
bd = flows_om[0][4]
hdbscan = 1
cluster = 0
newmasks,p,_,lab= omnipose.core.compute_masks(mu,dt,bd=bd,mask_threshold=mask_threshold,
                                               use_gpu=True,verbose=True,cluster=cluster, 
                                               flow_threshold=0., dim=3, hdbscan=hdbscan, 
                                                debug=1, eps=3, min_size=30, nclasses=dt.ndim+2,
                                              flow_factor=10)

In [ ]:
mnc = ncolor.label(newmasks)
viewer.add_labels(mnc,color=sinebow(mnc.max()))

np.unique(newmasks)
color = sinebow(newmasks.max())
# [color[l]]
selection = newmasks>0
sub_p = [p[i].ravel()[selection.ravel()] for i in range(p.shape[0])]
# sub_col = [color[l] for l in lab[selection.ravel()]]
sub_col = [color[l] for l in mnc[mnc>0]]
data = np.stack(sub_p,axis=-1)
viewer.add_points(np.round(data),size=1,symbol='disc',face_color=sub_col,edge_color=sub_col)

In [ ]:
6